In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [3]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model='gpt-5-nano',
    tools=[square_root]
)

subagent_2 = create_agent(
    model='gpt-5-nano',
    tools=[square]
)

## Calling subagents

In [4]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model='gpt-5-nano',
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [5]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [15]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='fd255179-9446-4a11-ab84-80f29327c625'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1308, 'prompt_tokens': 202, 'total_tokens': 1510, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1280, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cq1yVRi4ASXtoQv4OVm5dbbAX8IEm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b4c99-6dc7-7130-bc88-6bfef169ae18-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'call_dab6ZTsJhDMsTOy7aaF06d38', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 1308,

In [13]:
from langchain.messages import HumanMessage, ToolMessage, AIMessage

human = next(m for m in reversed(response["messages"]) if isinstance(m, HumanMessage))
tool  = next(m for m in reversed(response["messages"]) if isinstance(m, ToolMessage))
ai    = next(m for m in reversed(response["messages"]) if isinstance(m, AIMessage))

print(human.content)
print(tool.content)
print(ai.content)


What is the square root of 456?
sqrt(456.0) ≈ 21.3541565041
The square root of 456 is approximately 21.3541565041.  
(rounded: ~21.35)
